In [1]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import pandas as pd
import time
import itertools
from math import sqrt, log, atan, degrees

from skimage import feature
from skimage.transform import resize

import sys
sys.path.append('../')
from nifti import *

In [2]:
df = pd.read_csv('../data/ATR_GT_Training.csv', header=None, names=['file','label'])
df.file = df.file.map(lambda x: x.replace("'",''))
df['img'] = df.file.map(lambda x: nib.load('../data/' + str(x) + '.nii.gz'))

In [3]:
def quantize_image(data, n_levels):
    steps = np.max(data) / n_levels
    if steps == 0: steps = 1
    return (data / steps).astype(int)

In [4]:
def compute_glcm(I, Ng, dx, dy, dz, symmetric=True):
    '''gray level co-occurence matrix
    I: quantized image
    Ng: number of gray levels in quantized image
    https://en.wikipedia.org/wiki/Co-occurrence_matrix'''
    glcm = np.zeros((Ng,Ng))
    ix = itertools.product(range(Ng), range(Ng),
                           range(I.shape[0]), range(I.shape[1]), range(I.shape[2]))
    for i, j, x, y, z in ix:
        try:
            if I[x,y,z] == i and I[x+dx, y+dy, z+dz] == j:
                glcm[i,j] += 1
        except:
            pass
    if symmetric:
        glcm += glcm.T
    return glcm / np.sum(glcm)

def compute_glcm2(I, Ng, dx, dy, dz, symmetric=True):
    '''gray level co-occurence matrix
    I: quantized image
    Ng: number of gray levels in quantized image
    https://en.wikipedia.org/wiki/Co-occurrence_matrix'''
    def get_range(Ng, d):
        if d >= 0:
            return range(Ng-d)
        else:
            return range(-d,Ng)
    
    glcm = np.zeros((Ng,Ng))
    range_x = get_range(I.shape[0], dx)
    range_y = get_range(I.shape[1], dy)
    range_z = get_range(I.shape[2], dz)
    
    ix = itertools.product(range(Ng), range(Ng), range_x, range_y, range_y)
    for i, j, x, y, z in ix:
        if I[x,y,z] == i and I[x+dx, y+dy, z+dz] == j:
            glcm[i,j] += 1
    if symmetric:
        glcm += glcm.T
    return glcm / np.sum(glcm)

def compute_glcv(glcm, Ng):
    '''gray level co-occurence vector
    https://prism.ucalgary.ca/handle/1880/51900 page 24'''
    glcv = np.zeros((Ng))
    for i, j in itertools.product(range(Ng), range(Ng)):
        glcv[abs(i-j)] += glcm[i,j]
    return glcv

def compute_contrast(glcm, Ng):
    '''https://prism.ucalgary.ca/handle/1880/51900 page 30'''
    contrast = 0
    for i, j in itertools.product(range(Ng),range(Ng)):
        contrast += (i-j)**2 * glcm[i,j]
    return contrast

def compute_dissimilarity(glcm, Ng):
    '''https://prism.ucalgary.ca/handle/1880/51900 page 32'''
    dissimilarity = 0
    for i, j in itertools.product(range(Ng),range(Ng)):
        dissimilarity += abs(i-j) * glcm[i,j]
    return dissimilarity

def compute_homogeneity(glcm, Ng):
    '''https://prism.ucalgary.ca/handle/1880/51900 page 32'''
    homogeneity = 0
    for i, j in itertools.product(range(Ng),range(Ng)):
        homogeneity += glcm[i,j] / (1 + (i-j)**2)
    return homogeneity
        
def compute_angular_2nd_moment(glcm, Ng):
    '''https://prism.ucalgary.ca/handle/1880/51900 page 36'''
    a2m = 0
    for i, j in itertools.product(range(Ng),range(Ng)):
        a2m += glcm[i,j]**2
    return a2m

def compute_energy(glcm, Ng):
    '''https://prism.ucalgary.ca/handle/1880/51900 page 36'''
    return sqrt(compute_angular_2nd_moment)

def compute_entropy(glcm, Ng):
    '''https://prism.ucalgary.ca/handle/1880/51900 page 37'''
    entropy = 0
    for i, j in itertools.product(range(Ng),range(Ng)):
        if glcm[i,j] != 0:
            entropy += glcm[i,j] * -log(glcm[i,j])
    return entropy

def compute_glcm_mean(glcm, Ng, axis):
    '''https://prism.ucalgary.ca/handle/1880/51900 page 40'''
    mean = 0
    for i, j in itertools.product(range(Ng),range(Ng)):
        if axis == 0:
            mean += i * glcm[i,j]
        elif axis == 1:
            mean += j * glcm[i,j]
    return mean

def compute_glcm_variance(glcm, Ng, axis, mean=None):
    '''https://prism.ucalgary.ca/handle/1880/51900 page 41'''
    if mean == None:
        mean = compute_glcm_mean(glcm, Ng, axis)
        
    variance = 0
    for i, j in itertools.product(range(Ng),range(Ng)):
        if axis == 0:
            variance += (i-mean)**2 * glcm[i,j]
        elif axis == 1:
            variance += (j-mean)**2 * glcm[i,j]
    return variance

def compute_glcm_std_dev(glcm, Ng, axis, mean=None):
    '''https://prism.ucalgary.ca/handle/1880/51900 page 41'''
    return sqrt(compute_glcm_variance(glcm, Ng, axis))

def compute_correlation(glcm, Ng, u_i, u_j, var_i, var_j):
    '''https://prism.ucalgary.ca/handle/1880/51900 page 43'''
    correlation = 0
    for i, j in itertools.product(range(Ng),range(Ng)):
        correlation += glcm[i,j] * (i - u_i) * (j - u_j) / sqrt(var_i * var_j)
    return correlation

def compute_similarity(glcm, Ng):
    '''https://prism.ucalgary.ca/handle/1880/51900 page 43'''
    similarity = 0
    for i, j in itertools.product(range(Ng),range(Ng)):
        similarity += glcm[i,j] / (1 + abs(i-j))
    return similarity

In [5]:
Ng = 8
displacement_vectors = [[0,1,0],[-1,1,0],[-1,0,0],[-1,-1,0],
                       [0,1,-1],[0,0,-1],[0,-1,-1],[-1,0,-1],
                       [1,0,-1],[-1,1,-1],[1,-1,-1],[-1,-1,-1],[1,1,-1]]

texture_flat = []

start = time.time()
start100 = start
for i, img in enumerate(df.img):
    if i % 25 == 0 and i != 0:
        now = time.time()
        print('i = {:<10d}time for last 25 = {:<10.3g}total time = {:<10.3g}'.format(i,now-start100,now-start))
        start100 = time.time()
        
    data = img.get_fdata()
    data32 = resize(data, (16,16,16), mode='reflect', anti_aliasing=False)
    quantized_img = quantize_image(data32, Ng)
    
    texture_features = []
    for d in displacement_vectors[:]:
        dx,dy,dz = d
        start = time.time()
        glcm = compute_glcm2(quantized_img, Ng, dx, dy, dz)
        glcv = compute_glcv(glcm, Ng)
        angular_2nd_moment = compute_angular_2nd_moment(glcm, Ng)
        energy = sqrt(angular_2nd_moment)
        contrast = compute_contrast(glcm, Ng)
        dissimilarity = compute_dissimilarity(glcm, Ng)
        entropy = compute_entropy(glcm, Ng)
        mean = compute_glcm_mean(glcm, Ng, 0)
        var = compute_glcm_variance(glcm, Ng, 0, mean)
        std_dev = sqrt(var)
        correlation = compute_correlation(glcm, Ng, mean, mean, var, var)
        similarity = compute_similarity(glcm, Ng)
        features = list(glcv) + [angular_2nd_moment, energy, contrast, dissimilarity,
                               entropy, mean, var, std_dev, correlation, similarity]
        texture_features.append(features)
    texture_features = np.array(texture_features)
    texture_flat.append(list(np.resize(texture_features, np.product(texture_features.shape))))

i = 25        time for last 25 = 12.1      total time = 0.0342    


C:\Users\coone\AppData\Local\Temp\ipykernel_4080\2509639181.py:122: RuntimeWarning: invalid value encountered in scalar divide
  correlation += glcm[i,j] * (i - u_i) * (j - u_j) / sqrt(var_i * var_j)


i = 50        time for last 25 = 12.1      total time = 0.0347    
i = 75        time for last 25 = 12.1      total time = 0.0347    
i = 100       time for last 25 = 12.1      total time = 0.0347    
i = 125       time for last 25 = 12.1      total time = 0.0337    
i = 150       time for last 25 = 12.1      total time = 0.0337    
i = 175       time for last 25 = 12        total time = 0.0347    
i = 200       time for last 25 = 12        total time = 0.0342    
i = 225       time for last 25 = 12        total time = 0.0337    
i = 250       time for last 25 = 12        total time = 0.0342    
i = 275       time for last 25 = 12        total time = 0.0342    
i = 300       time for last 25 = 11.9      total time = 0.0337    
i = 325       time for last 25 = 12        total time = 0.0337    
i = 350       time for last 25 = 11.9      total time = 0.0342    
i = 375       time for last 25 = 11.9      total time = 0.0342    
i = 400       time for last 25 = 11.9      total time = 0.0342

In [7]:
np.save('features_texture.npy',np.array(texture_flat))